In [ ]:
import pandas as pd
import requests
import json
import io
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import datetime
import ast
import urllib

In [ ]:
# ! pip install bs4

In [38]:
PCRsFileName = '../../data/pcrs-20200406.xlsx'
PCRs = pd.read_excel(PCRsFileName)
print(PCRs.columns)

Index(['Unnamed: 0', 'Countries', 'Download Link', 'MODALITY_FR_DUMP',
       'Milestone Approval Date', 'Milestone Effectivity Date',
       'Milestone Signing Date', 'Modalities', 'Month', 'Month Year',
       'Project Number', 'Safeguard Categories', 'Sectors',
       'Source Of Funding', 'Themes', 'Title', 'UNQ_MODALITY_FR_DUMP', 'Year',
       'id'],
      dtype='object')


In [40]:
currentProjectNumbers = PCRs["Project Number"].tolist()
currentDownloadLinks = PCRs["Download Link"].tolist()
currentIDs = PCRs["id"].tolist()

In [77]:
domain = 'https://www.adb.org/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
nPages = 65
startPage = 0
x = datetime.datetime.now()
YYYYmmdd = x.strftime("%Y%m%d")
folderName = "../../data/adb-pcr-" + YYYYmmdd + "/"


In [ ]:
! mkdir "$folderName"

In [78]:
maxID = max(currentIDs)
count = 0
titles = []
ids = []
downloadLinks = []
projectNumbers = []
allCountries = []
allSectors = []
allThemes = []
fileNames = []
allYears = []
allMonths = []
allMonthYear = []

for page in range(startPage, nPages):
    print("Page: " + str(page))
    url = domain + 'projects/documents/doctype/Project%252FProgram%20Completion%20Reports?page=' + str(page)
    r = requests.get(url, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    itemTitles = soup.findAll("div", {"class": "item-title"})
    for title in itemTitles:

        # Title
        titleText = title.find("a").text
        # print("Current Document: " + titleText, ", Count: " + str(count))
        count = count + 1

        # if(count == 161):
        # Download Link and id
        documentLink = domain + title.find("a").get('href')
        r2 = requests.get(documentLink, headers=headers)
        html2 = r2.text
        soup2 = BeautifulSoup(html2, 'html.parser')
        asideRegion = soup2.find("aside", {"class": "region"})
        downloadLink = ''
        if(asideRegion):
            downloadLink = asideRegion.find("a").get('href')
        

        # Project Number
        articleTags = soup2.find("div", {"class": "article-tags"})
        fieldItems = articleTags.findAll("ul", {"class": "field-items"})
        projectNumberField = fieldItems[0].find("li", {"class": "field-item"})
        projectNumber = projectNumberField.find("a").text
        
        # if projectNumber in currentProjectNumbers:
        #     print("Already scraped")
        #     continue
        
        # print("New PCR", str(maxID), projectNumber )
        # maxID = maxID + 1
        titles.append(titleText)
        # ids.append(maxID)
        projectNumbers.append(projectNumber)
        # downloadLinks.append(downloadLink)

        # Month Year
        # dateDisplay = soup2.find("span", {"class": "date-display-single"})
        # allMonthYear.append(dateDisplay.text)
        # allMonths.append(dateDisplay.text.split(" ")[0])
        # allYears.append(dateDisplay.text.split(" ")[1])
        
        # Sector
        sectorFound = True
        representativeSector = ''
        sectors = []
        if(len(fieldItems)>2):
            sectorFields = fieldItems[2].findAll("li", {"class": "field-item"})
            for sectorField in sectorFields:
                sectors.append(sectorField.find("a").text)
            if(len(sectors)):
                representativeSector = sectors[0]
        else:
            sectorFound = False

        representativeTheme = ''
        representativeSector = ''
        projectNumberLink = projectNumberField.find("a").get('href')
        r3 = requests.get(domain + projectNumberLink + '#project-pds')
        html3 = r3.text
        soup3 = BeautifulSoup(html3, 'html.parser')
        themes = []
        if(soup3.find("table", {"class": "pds"})):
            pdsTableTrs = soup3.find("table", {"class": "pds"}).findAll("tr")
            for tr in pdsTableTrs:
                trtds = tr.findAll("td")
                if(trtds):
        # Theme
                    # if(trtds[0].text == 'Strategic Agendas'):
                    #     if(len(trtds)>1):
                    #         themesRaw = str(trtds[1]).split(' <br/>')
                    #         themesRaw[0] = themesRaw[0][4:]
                    #         themes = themesRaw[:-1]
                    #         if(themes):
                    #             representativeTheme = themes[0]
        # Sector (2nd chance)
                    if not sectorFound:
                        if(trtds[0].text == 'Sector / Subsector'):
                            if(len(trtds)>1):
                                if(trtds[1].find("p")):
                                    sector = trtds[1].find("p").text.split("/")[0].rstrip()
                                    sectors.append(sector)
                                    if(sectors):
                                        representativeSector = sectors[0]
                                else:
                                    print(documentLink)
        if sectors == []:
            print(documentLink)
        allSectors.append(sectors)
        # print(sectors)

        # Country
        # representativeCountry = ''
        # countries = []
        # if(len(fieldItems)>1):
        #     countryFields = fieldItems[1].findAll("li", {"class": "field-item"})
        #     for countryField in countryFields:
        #         countries.append(countryField.find("a").text)
        #     representativeCountry = countries[0]
        # allCountries.append(countries)


        # File
        # startFileName = 0
        # for i in range(len(downloadLink)-1, 0, -1):
        #     if downloadLink[i] == '/':
        #         startFileName = i + 1
        #         break

        # fileName = downloadLink[startFileName:]
        # fileNames.append(fileName)
        # if(downloadLink and downloadLink[0] != '/'):
        #     r4 = requests.get(downloadLink)
        #     with open(folderName + fileName, 'wb') as f:
        #         f.write(r4.content)
        # else:
        #     with open(folderName + fileName + '-invalid.txt', 'w') as f:
        #         f.write('Invalid file')

        # Write to spreadsheet
        if count % 50 == 0:
            df = pd.DataFrame()
            # df['id'] = ids
            df['Title'] = titles
            # df['Download Link'] = downloadLinks
            df['Project Number'] = projectNumbers
            # df['Month Year'] = allMonthYear
            # df['Month'] = allMonths
            # df['Year'] = allYears
            # df['Countries'] = allCountries
            df['Sectors'] = allSectors
            # df['Themes'] = allThemes
            # df['File Names'] = fileNames
            df.to_excel(folderName + 'pcrs-new-data-partial-' + str(count) + '.xlsx')
            print(count)
    #     break
    # break

df = pd.DataFrame()
# df['id'] = ids
df['Title'] = titles
# df['Download Link'] = downloadLinks
df['Project Number'] = projectNumbers
# df['Month Year'] = allMonthYear
# df['Month'] = allMonths
# df['Year'] = allYears
# df['Countries'] = allCountries
df['Sectors'] = allSectors
# df['Themes'] = allThemes
# df['File Names'] = fileNames
df.to_excel(folderName + 'pcrs-new-data.xlsx')

ps://www.adb.org//projects/documents/emergency-infrastructure-rehabilitation-project-phase-2
Page: 37
https://www.adb.org//projects/documents/national-highways-project
750
https://www.adb.org//projects/documents/grains-sector-development-program
https://www.adb.org//projects/documents/forestry-sector-project-0
Page: 38
Page: 39
800
Page: 40
https://www.adb.org//projects/documents/primary-roads-restoration-project-0
https://www.adb.org//projects/documents/xieng-khouang-road-improvement-project
Page: 41
https://www.adb.org//projects/documents/clark-area-municipal-development-project
https://www.adb.org//projects/documents/regional-municipal-development-project
https://www.adb.org//projects/documents/regional-municipal-development-project
https://www.adb.org//projects/documents/north-central-province-rural-development-project
https://www.adb.org//projects/documents/third-livestock-development-project
Page: 42
https://www.adb.org//projects/documents/southern-yunnan-road-development-project

In [ ]:
df = pd.read_excel(folderName + ".xlsx")

In [ ]:
df = pd.DataFrame()
df['Title'] = titles
df['Project Number'] = projectNumbers
df['Countries'] = allCountries
df['Sectors'] = allSectors
df['Themes'] = allThemes
df['Download Link'] = downloadLinks
df.to_excel(folderName + 'pcrs-new-data.xlsx')


In [26]:
df = pd.read_excel(folderName + 'pcrs-new-data.xlsx')
df['MODALITY_FR_DUMP'] = "[]"
df['UNQ_MODALITY_FR_DUMP'] = "set()"
df['Milestone Approval Date'] = ""
df['Milestone Effectivity Date'] = ""
df['Milestone Signing Date'] = ""
df['Modalities'] = "[]"
df['Safeguard Categories'] = ""

In [36]:
print(df.columns)
print(PCRs.columns)

Index(['id', 'Title', 'Download Link', 'Project Number', 'Month Year', 'Month',
       'Year', 'Countries', 'Sectors', 'Themes', 'MODALITY_FR_DUMP',
       'UNQ_MODALITY_FR_DUMP', 'Milestone Approval Date',
       'Milestone Effectivity Date', 'Milestone Signing Date', 'Modalities',
       'Safeguard Categories'],
      dtype='object')
Index(['id', 'Title', 'Project Number', 'Countries', 'Sectors', 'Themes',
       'Month Year', 'Month', 'Year', 'Modalities', 'Source Of Funding',
       'Safeguard Categories', 'Milestone Approval Date',
       'Milestone Signing Date', 'Milestone Effectivity Date', 'Download Link',
       'MODALITY_FR_DUMP', 'UNQ_MODALITY_FR_DUMP'],
      dtype='object')


In [35]:
df = df.drop(['File Names'], axis=1)
PCRs = PCRs.drop(['Unnamed: 0'], axis=1)

In [37]:
PCRs = PCRs.append(df)
PCRs.to_excel(folderName + 'pcrs-data.xlsx')

/Users/jraramirez/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
